In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as soup 
import re
import pandas as pd
import numpy as np
import time
import json
import random

In [2]:
browser = Browser('chrome')
city = "Ahmedabad"
target_cars = 5000
cars_collected = 0
total_pages = 250 

In [ ]:
# 1. Open a text file in 'a' (append) mode
with open("car_links.txt", "a") as f:

    for page_num in range(1, total_pages + 1):
        if page_num == 1:
            url = f"https://www.cardekho.com/used-cars+in+{city}"
        else:
            url = f"https://www.cardekho.com/used-cars+in+{city}/page-{page_num}"
        
        browser.visit(url)
        time.sleep(2) 
        
        browser.execute_script("window.scrollTo(0, 1000);")
        time.sleep(1)

        current_soup = soup(browser.html, 'html.parser')
        
        page_links_found = 0
        for link in current_soup.find_all('a', href=True):
            href = link['href']
            
            if 'used-car-details' in href:
                # Build the full URL
                full_url = f"https://www.cardekho.com{href}" if href.startswith('/') else href
                
                # 2. Write the link to the text file immediately
                f.write(full_url + "\n")
                
                cars_collected += 1
                page_links_found += 1

        print(f"Page {page_num}: Saved {page_links_found} links. Total: {cars_collected}")

        # Stop if target reached
        if cars_collected >= target_cars:
            print("Target 5000 reached!")
            break

print("All links are now saved in car_links.txt")

Page 1: Saved 28 links. Total: 28
Page 2: Saved 28 links. Total: 56
Page 3: Saved 28 links. Total: 84
Page 4: Saved 28 links. Total: 112
Page 5: Saved 28 links. Total: 140
Page 6: Saved 28 links. Total: 168
Page 7: Saved 28 links. Total: 196
Page 8: Saved 28 links. Total: 224
Page 9: Saved 28 links. Total: 252
Page 10: Saved 28 links. Total: 280
Page 11: Saved 28 links. Total: 308
Page 12: Saved 28 links. Total: 336
Page 13: Saved 28 links. Total: 364
Page 14: Saved 28 links. Total: 392
Page 15: Saved 28 links. Total: 420
Page 16: Saved 28 links. Total: 448
Page 17: Saved 28 links. Total: 476
Page 18: Saved 28 links. Total: 504
Page 19: Saved 28 links. Total: 532
Page 20: Saved 28 links. Total: 560
Page 21: Saved 28 links. Total: 588
Page 22: Saved 28 links. Total: 616
Page 23: Saved 28 links. Total: 644
Page 24: Saved 28 links. Total: 672
Page 25: Saved 28 links. Total: 700
Page 26: Saved 28 links. Total: 728
Page 27: Saved 28 links. Total: 756
Page 28: Saved 28 links. Total: 784
Page

## Main Extraction 

In [3]:
def get_browser():
    return Browser('chrome')

# 1. Load your links
with open("car_links.txt", "r") as f:
    all_links = [line.strip() for line in f.readlines()]

output_file = f"car_dataset_{city}.json"
browser = get_browser()

# 2. Loop through each link
for index, link in enumerate(all_links):
    try:
        print(f"Scraping {index+1}/{len(all_links)}: {link}")
        
        # Visit the link once for all data
        browser.visit(link)
        
        # Human-like delay and scroll to trigger dynamic content
        time.sleep(random.uniform(.5, 1))
        browser.execute_script("window.scrollTo(0, 600);")
        time.sleep(0.5)

        # 3. Expansion Logic (Clicking 'View all Specifications')
        try:
            view_all_spec_btn = browser.find_by_text('View all Specifications')
            if view_all_spec_btn:
                browser.execute_script("arguments[0].click();", view_all_spec_btn.first._element)
                print("Expanded specifications.")
                time.sleep(0.6) # Wait for table to render
        except Exception:
            # Skip if button is missing or already expanded
            pass

        # 4. Scrape with BeautifulSoup
        page_soup = soup(browser.html, 'html.parser')
        car_data = {"url": link}
        
        # --- PRICE EXTRACTION ---
        price_div = page_soup.find('div', class_='vehiclePrice')
        if price_div:
            price_span = price_div.find('span')
            if price_span:
                car_data["Price"] = price_span.get_text(strip=True)

        # --- SPECIFICATIONS EXTRACTION ---
        spec_items = page_soup.find_all('li', class_='gsc_col-xs-12')
        for item in spec_items:
            label_tag = item.find('div', class_='label')
            value_tag = item.find('span', class_='value-text')
            if label_tag and value_tag:
                label = label_tag.get_text(strip=True)
                value = value_tag.get_text(strip=True)
                car_data[label] = value

        # 5. Save if valid data was found
        if len(car_data) > 1:
            with open(output_file, "a") as out:
                out.write(json.dumps(car_data) + "\n")
            print(f"Saved: {car_data.get('Price', 'N/A')} and {len(car_data)-2} specs.")
        else:
            print(f"No data found for: {link}")

    except Exception as e:
        print(f"Serious error at {link}: {e}")
        # Refresh browser instance on crash
        browser.quit()
        browser = get_browser()
        time.sleep(.75)
        continue

browser.quit()

Scraping 1/5108: https://www.cardekho.com/used-car-details/used-Hyundai-verna-crdi-16-sx-option-cars-Lucknow_391b86cb-1f11-477e-b446-3dd525c8b095.htm?adId=20601&adType=41
Expanded specifications.
Saved: ₹8 Lakh and 47 specs.
Scraping 2/5108: https://www.cardekho.com/used-car-details/used-Volkswagen-tiguan-20-tsi-elegance-cars-New-Delhi_c33c0387-3a39-494d-98f5-eb26c891d263.htm
Expanded specifications.
Saved: ₹29.90 Lakh and 48 specs.
Scraping 3/5108: https://www.cardekho.com/buy-used-car-details/used-Maruti-swift-dzire-zxi-cars-Kochi_e4614bdc-af71-42bf-b20e-2858244ed5ad.htm
Expanded specifications.
Saved: ₹3.60 Lakh and 47 specs.
Scraping 4/5108: https://www.cardekho.com/used-car-details/used-Bmw-3-series-gt-luxury-line-cars-Bangalore_3a83bb05-ccd7-4173-9315-7c3341b7c8e5.htm
Expanded specifications.
Saved: ₹24.75 Lakh and 50 specs.
Scraping 5/5108: https://www.cardekho.com/used-car-details/used-Toyota-corolla-altis-d-4d-g-cars-Panchkula_6dfeed15-c1b8-420f-9a08-3d4447e10297.htm?adId=1784

There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache
Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)


Scraping 3730/5108: https://www.cardekho.com/used-car-details/used-Maruti-wagon-r-lxi-bs-iv-cars-Ahmedabad_295fbbcb-b185-46b4-b021-2202e1389a1a.htm
No data found for: https://www.cardekho.com/used-car-details/used-Maruti-wagon-r-lxi-bs-iv-cars-Ahmedabad_295fbbcb-b185-46b4-b021-2202e1389a1a.htm
Scraping 3731/5108: https://www.cardekho.com/used-car-details/used-Maruti-s-cross-ddis-200-zeta-cars-Ahmedabad_5f0b9bdb-9153-4ea5-b6ed-9637f4d2a029.htm
Expanded specifications.
Saved: ₹5 Lakh and 50 specs.
Scraping 3732/5108: https://www.cardekho.com/used-car-details/used-Hyundai-grand-i10-sportz-cars-Ahmedabad_edc56ff5-2fbe-4d20-a714-dadcc809eaa0.htm
Expanded specifications.
Saved: ₹3.50 Lakh and 52 specs.
Scraping 3733/5108: https://www.cardekho.com/used-car-details/used-Honda-amaze-v-petrol-bsiv-cars-Ahmedabad_4d63b0c3-31fa-4f57-ac5f-c877fb36fec0.htm
Expanded specifications.
Saved: ₹4.70 Lakh and 51 specs.
Scraping 3734/5108: https://www.cardekho.com/buy-used-car-details/used-Hyundai-grand-i10

In [ ]:
def convert_json_to_cleaned_csv(input_json, output_csv):
    try:
        df = pd.read_json(input_json, lines=True)
        
        if df.empty:
            print(f"No data found in {input_json}")
            return

        df.columns = [col.strip() for col in df.columns]

        df.to_csv(output_csv, index=False, encoding='utf-8-sig')
        print(f"Successfully converted {len(df)} rows to {output_csv}")

    except Exception as e:
        print(f"Error during conversion: {e}")

convert_json_to_cleaned_csv(f'car_dataset_{city}.json', f'car_dataset_{city}.csv')

Successfully converted 5105 rows to car_dataset_ahmedabad.csv
